In [1]:
# agent workflow (where agents deal wit heach other)
# use of TOOLS to call functions
# agen collaboration via TOOLS and HANDOFFS

In [59]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import os

In [5]:
load_dotenv(override=True)

True

In [132]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("hello@priyanshukhandelwal.com")  # Change to your verified sender
    to_email = To("udemy@priyanshukhandelwal.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [22]:
# lets see if email are working
def send_email():
    sg_client = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("hello@priyanshukhandelwal.com")
    to_email = To("udemy@priyanshukhandelwal.com")
    subject = "Test Email from SendGrid"
    content = Content("text/plain", "This is a test email sent using SendGrid.")
    mail = Mail(from_email, to_email, subject, content).get()
    print(mail)
    response = sg_client.send(mail)
    print(f"Email sent with status code: {response.status_code}")
send_email()

{'from': {'email': 'hello@priyanshukhandelwal.com'}, 'subject': 'Test Email from SendGrid', 'personalizations': [{'to': [{'email': 'udemy@priyanshukhandelwal.com'}]}], 'content': [{'type': 'text/plain', 'value': 'This is a test email sent using SendGrid.'}]}
Email sent with status code: 202


# Setting up agentic flow with 3 agents

In [26]:
instructions1 = '''You are sales agent working for DeepAI, 
    a company that provides help to other software companies by enhancing their
    legacy systems with current AI approaches, powered by AI. You write professional, serious
    cold emails '''

instructions2 = '''You are humorous funny sales agent working for DeepAI, 
    a company that provides help to other software companies by enhancing their
    legacy systems with current AI approaches, powered by AI. You write professional, likely to get response
    cold emails '''

instructions3 = '''You are busy sales agent working for DeepAI, 
    a company that provides help to other software companies by enhancing their
    legacy systems with current AI approaches, powered by AI. You write professional and concise emails
'''

agent1 = Agent(name="Agent 1", instructions=instructions1, model="gpt-4o-mini")
agent2 = Agent(name="Agent 2", instructions=instructions2, model="gpt-4o-mini")
agent3 = Agent(name="Agent 3", instructions=instructions3, model="gpt-4o-mini")


In [37]:
runner = Runner()
result = await runner.run(agent3, "Write a cold email")

In [38]:
print(result.final_output)

Subject: Enhance Your Legacy Systems with Advanced AI Solutions

Hi [Recipient's Name],

I hope this message finds you well. I’m [Your Name] from DeepAI, where we specialize in revitalizing legacy systems with cutting-edge AI technologies.

Many companies face challenges with outdated systems, which can hinder efficiency and innovation. Our solutions can seamlessly integrate advanced AI capabilities, enabling your team to optimize operations and enhance user experiences.

I’d love to schedule a brief call to discuss how we can help [Recipient's Company] leverage AI to drive growth and efficiency. Are you available for a chat this week?

Looking forward to your response.

Best,  
[Your Name]  
[Your Job Title]  
DeepAI  
[Your Phone Number]  
[Your Email Address]  
[Your Website]  


In [66]:
result = Runner.run_streamed(agent3, "Write a cold email only 30 words")
async for event in result.stream_events():
    # moving from left to right
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

# runner.run
# runner.run_streamed
# event.type
# multiple instances of event data

Subject: Enhance Your Legacy Systems with AI

Hi [Name],

Unlock your software’s potential with DeepAI. We specialize in modernizing legacy systems using advanced AI solutions. Let’s discuss how we can help your business thrive.

Best,  
[Your Name]  
DeepAI  

In [68]:
message = "Write a cold email max words allowed is 100"

with trace("Parallel Code Emails"):
    results = await asyncio.gather(
        Runner.run(agent1, message),
        Runner.run(agent2, message),
        Runner.run(agent3, message)
    )

In [70]:
outputs = [result.final_output for result in results]

In [71]:
for output in outputs:
    print(output)
    print("-----")


Subject: Enhance Your Legacy Systems with AI

Dear [Recipient's Name],

I hope this message finds you well. At DeepAI, we specialize in transforming legacy systems into modern, AI-powered solutions. Our tailored services can help streamline your operations, improve efficiency, and drive innovation within your organization.

I would love to discuss how we can assist [Company Name] in leveraging AI to stay competitive in today’s market. 

Looking forward to the opportunity to connect.

Best regards,  
[Your Name]  
[Your Position]  
DeepAI  
[Your Phone Number]  
[Your Email Address]  
-----
Subject: Let’s Upgrade Your Legacy Systems—With a Laugh! 😂

Hi [Recipient's Name],

Is your legacy system feeling as outdated as a flip phone in 2023? At DeepAI, we specialize in supercharging those old workhorses with cutting-edge AI—no time machine required!

Let’s chat about how we can turn your dinosaur into a data-driven dynamo. I promise, it’ll be more fun than debugging!

Looking forward to co

In [72]:
mail_picker = Agent(
    name = "mail-picker",
    instructions = "Your duty is to pick the best email from the list of emails. You should select only one email to which you are likely to respond best. Don't give any explanation; just respond with the selected email",
    model = "gpt-4o-mini"
)

In [79]:
runner_selector = Runner()
result = await runner_selector.run(mail_picker, '_______'.join(outputs)  )
print(result.final_output)

Subject: Enhance Your Legacy Systems with AI Solutions

Hi [Recipient's Name],

I hope this message finds you well. I'm [Your Name] from DeepAI, where we specialize in upgrading legacy systems with cutting-edge AI technologies. Our solutions can enhance efficiency, reduce costs, and improve customer satisfaction.

I'd love to discuss how we can support your business in leveraging AI. Are you available for a quick call this week?

Best regards,  
[Your Name]  
[Your Position]  
DeepAI  
[Your Contact Information]  


In [80]:
# Lets put it all together
# 3 agents to generate 3 types of email
# 1 agent to select the best email

instructions1 = instructions1
instructions2 = instructions2
instructions3 = instructions3
instructions_mail_picker = "Your duty is to pick the best email from the list of emails. You should select only one email to which you are likely to respond best. Don't give any explanation; just respond with the selected email"

agent1 = Agent(name="Agent 1", instructions=instructions1, model="gpt-4o-mini")
agent2 = Agent(name="Agent 2", instructions=instructions2, model="gpt-4o-mini")
agent3 = Agent(name="Agent 3", instructions=instructions3, model="gpt-4o-mini")
mail_picker_agent = Agent(name = 'Agent mail picker', instructions = instructions_mail_picker, model = "gpt-4o-mini")

with trace("Generate and pick cold email"):
    results = await asyncio.gather(
        Runner.run(agent1, "Write a cold email"),
        Runner.run(agent2, "Write a cold email"),
        Runner.run(agent3, "Write a cold email"),
    )
    outputs = [result.final_output for result in results]

    final_combined_output = "____________".join(outputs)

    selected_email = await Runner.run(mail_picker_agent, final_combined_output)

    print(selected_email.final_output)


Subject: Upgrade Your Legacy System... Before It Becomes a Museum Exhibit! 🏛️

Hi [Recipient's Name],

I hope this email finds you well—assuming your legacy systems haven’t taken over your entire office space yet! 

At DeepAI, we specialize in transforming those “classic” systems from the days when floppy disks were considered cutting-edge. You know, the ones that require a dedicated IT team just to start up? We use cutting-edge AI technology to breathe new life into them, making them as smart as your latest smartphone (minus the selfies, of course). 

Imagine your archaic system smoothly integrating AI powers that can handle tasks faster than you can say “software upgrade” while still maintaining the peace of mind that your data is safe. Sounds dreamy, right?

I'd love to chat about how we can help your software get an upgrade without a complete overhaul (and save you from explaining to your boss why your system looks like something out of a sci-fi movie gone wrong).

Can we schedule 

# Using Tools

In [82]:
agent1 = Agent(name="Professional agent", instructions=instructions1, model="gpt-4o-mini")
agent2 = Agent(name="Funny agent", instructions=instructions2, model="gpt-4o-mini")
agent3 = Agent(name="Concise agent", instructions=instructions3, model="gpt-4o-mini")

In [ ]:
@function_tool
def send_email(body):
    sg_client = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("hello@priyanshukhandelwal.com")
    to_email = To("udemy@priyanshukhandelwal.com")
    content = Content("text/plain", body)
    subject = "Test Email from SendGrid"
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg_client.send(mail)
    print(f"Email sent with status code: {response.status_code}")


In [85]:
send_email('hello pk pk pk')

Email sent with status code: 202


-------------

## Understanding decorators

In Python, a decorator is a special type of function that can modify or extend the behavior of another function. A decorator is a function that **takes another function as an argument** and returns a new function that "wraps" the original function.

In [86]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


In this example, the my_decorator function is a decorator that takes the say_hello function as an argument. The my_decorator function returns a new function called wrapper, which "wraps" the original say_hello function.

When we call say_hello(), we're actually calling the wrapper function returned by the my_decorator. The wrapper function calls the original say_hello function, but also adds some extra behavior before and after the call.

Decorators are often used to:

 - Add logging or debugging functionality to a function
 - Implement authentication or authorization checks
 - Measure the execution time of a function
 - Cache the results of a function
 - And many other use cases!

 The @ symbol is used to indicate that a function is a decorator. When we use the @ symbol before a function, it's equivalent to calling the decorator function with the original function as an argument.
 
----------

In [91]:
import logging
import functools

# Set up logging
logging.basicConfig(level=logging.INFO)

def log_calls(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        logging.info(f"Calling {func.__name__} with arguments {args} and {kwargs}")
        result = func(*args, **kwargs)
        logging.info(f"{func.__name__} returned {result}")
        return result
    return wrapper

@log_calls
def add(a, b):
    return a + b

result = add(2, 3)

add(10,200)

INFO:root:Calling add with arguments (2, 3) and {}
INFO:root:add returned 5
INFO:root:Calling add with arguments (10, 200) and {}
INFO:root:add returned 210


210

## Using Tools Continue..

#### READ THIS:
Yes, that makes sense. By adding the str type hint to the body parameter, you're explicitly defining the schema for the send_email function.

In this case, the error message was indicating that the schema for the send_email function was missing a type key. By adding the str type hint, you're providing the necessary information to define the schema for the body parameter.

This is a good example of how type hints can be used to provide additional metadata about the structure and schema of your code. In this case, the type hint is being used to define the schema for the send_email function, which is then used by the underlying framework or library to validate the input data.

It's also worth noting that this is a good example of how a small change can make a big difference in the behavior of your code. In this case, adding a single type hint (str) was enough to resolve the error and get the code working correctly.

In [ ]:
@function_tool
def send_email(body: str): ## Adding str is very very very important
    sg_client = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("hello@priyanshukhandelwal.com")
    to_email = To("udemy@priyanshukhandelwal.com")
    content = Content("text/plain", body)
    subject = "Test Email from SendGrid"
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg_client.send(mail)
    print(f"Email sent with status code: {response.status_code}")

In [106]:
send_email

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11ee318a0>, strict_json_schema=True, is_enabled=True)

**^^^^^^ This is how functions are converted to TOOL ^^^^^^^^**

In [107]:
tool1 = agent1.as_tool(tool_name='tool_agent_1',
                        tool_description = 'Write a cold email',)

**^^^^^^^This is how agents are converted to TOOL^^^^^^^^^^**

### Using all together to crreate tool list and converting all agents to tools.
Because ultimately our email_picker is using three agents indirectly as tool only

In [108]:
agent1

Agent(name='Professional agent', instructions='You are sales agent working for DeepAI, \n    a company that provides help to other software companies by enhancing their\n    legacy systems with current AI approaches, powered by AI. You write professional, serious\n    cold emails ', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [109]:
description_for_mail_generation = "Write a cold email"
tool1 = agent1.as_tool(tool_name='agent1', tool_description=description_for_mail_generation)
tool2 = agent2.as_tool(tool_name='agent2', tool_description=description_for_mail_generation)
tool3 = agent3.as_tool(tool_name='agent3', tool_description=description_for_mail_generation)
send_email_tool = send_email # which is essentially a function tool decorated with @function_tool
tools = [tool1, tool2, tool3, send_email_tool]

## Now its time for our Sales Manager to email the client (select from agents and send using tools) 

In [110]:
send_email_tool == send_email

True

In [111]:
print(sales_manager_agent)

Agent(name='Sales Manager', instructions='\nYou are sales manager working for DeepAI. You use the tools give to you to generate cold emails.\nYou never generate sales emails by yourself; you always use tools.\nYou try all 3 agent_tools once before choosing the best one. You can use tools iteratively untill you find a best email to send.\nYou pick the single best email and use send_email_tool to send the best email to the user.\n', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[FunctionTool(name='agent1', description='Write a cold email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}

In [112]:
sales_manager_instructions = '''
You are sales manager working for DeepAI. You use the tools give to you to generate cold emails.
You never generate sales emails by yourself; you always use tools.
You try all 3 agent_tools once before choosing the best one. You can use tools iteratively untill you find a best email to send.
You pick the single best email and use send_email_tool to send the best email to the user.
'''

sales_manager_agent = Agent(name="Sales Manager", instructions=sales_manager_instructions, model="gpt-4o-mini", tools=tools)
message = 'Send a col email addressed to Dear Senior Head of AI'
with trace("Generate, select and send email: Sales Manager Agent"):
    result = await Runner.run(sales_manager_agent, message)
    print(result.final_output)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Email sent with status code: 202


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


The cold email has been sent successfully! If there's anything else you need, feel free to ask.


In [ ]:
# test_agent = Agent(name='query_resolver', instructions='resolve_query', model='gpt-4o-mini')
# result = await Runner.run(test_agent, 'convert 800 crore inr to usd, and give me only final answer')
# print(result.final_output)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


800 crore INR is approximately 96 million USD.


# Using handoffs

Its important to know difference in handoffs and agent_as_tools.
- With tools, control passes back
- With handoffs, control passes across

In [117]:
# Build an emailer_agent that can generate email and send it
print('''
- 3 agents/tools to generate email
- 1 agent/tool to select the best email
- 1 agent/tool to decide the subject
- 1 agent/tool to convert text to html
- 1 tool to send the email
''')


- 3 agents/tools to generate email
- 1 agent/tool to select the best email
- 1 agent/tool to decide the subject
- 1 agent/tool to convert text to html
- 1 tool to send the email



In [126]:
from typing import Dict


In [128]:
send_email_tool

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11ee318a0>, strict_json_schema=True, is_enabled=True)

In [130]:
instruction_for_agent_1 = '''
You work for PKAI company, you are an expert in writing professional cold emails.'''
instruction_for_agent_2 = '''
You work for PKAI company you are an expert in writing humorous cold emails'''
instruction_for_agent_3 = '''
You work for PKAI company you are an expert in writing concise cold emails'''
instruction_for_subject_writer = '''
You are an expert in writing subject for emails'''
instruction_for_html_writer = '''
You are an expert in converting text to html'''

@function_tool
def send_email(subject:str, body:str) -> Dict[str, str]:
    sg_client = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("hello@priyanshukhandelwal.com")
    to_email = To("udemy@priyanshukhandelwal.com")
    content = Content("text/html", body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg_client.send(mail)
    print(f"Email sent with status code: {response.status_code}")

agent_1_tool = Agent(name='agent_1', instructions=instruction_for_agent_1, model='gpt-4o-mini').as_tool(tool_name='agent_1_tool', tool_description='Write a cold email') 
agent_2_tool = Agent(name='agent_2', instructions=instruction_for_agent_2, model='gpt-4o-mini').as_tool(tool_name='agent_2_tool', tool_description='Write a cold email')
agent_3_tool = Agent(name='agent_3', instructions=instruction_for_agent_3, model='gpt-4o-mini').as_tool(tool_name='agent_3_tool', tool_description='Write a cold email')
subject_writer_tool = Agent(name='subject_writer', instructions=instruction_for_subject_writer, model='gpt-4o-mini').as_tool(tool_name='subject_writer_tool', tool_description='Write a subject')
html_writer_tool = Agent(name='html_writer', instructions=instruction_for_html_writer, model='gpt-4o-mini').as_tool(tool_name='html_writer_tool', tool_description='Convert text to html')
send_email_tool = send_email
tools = [agent_1_tool, agent_2_tool, agent_3_tool, subject_writer_tool, html_writer_tool, send_email_tool]

sales_manager_instructions = '''
You are a sales manager for PKAI. You use the tools give to you to generate cold emails.
You never generate sales emails by yourself; you always use tools.
You try all 3 agent_tools once before choosing the best one. You can use tools iteratively untill you find a best email to send.
You pick the single best email. Then you decide the subject of the email using subject_writer tool.Then you use html_writer tool to convert the text to html.
Only use the subject decided by subject_writer_tool, remove if there is any additional subject in the email.
Then you use send_email_tool to send the email to the user.
'''

sales_manager_agent = Agent(name='Sales Manager', instructions=sales_manager_instructions, model='gpt-4o-mini', tools=tools)
message = 'Send a cold email addressed to Dear Senior Head of AI'
with trace('Generate, select body, convert to html and send email: Sales Manager Agent '):
    result = await Runner.run(sales_manager_agent, message)
    print(result.final_output)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Email sent with status code: 202


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


The cold email has been successfully sent to the Senior Head of AI with the subject "Exploring AI Collaboration Opportunities." If you need any more assistance, feel free to ask!


In [131]:
# research how you can have SendGrid call a Callback webhook when a user replies to an email, Then have the SDR respond to keep the conversation going

# TODO : Priyanshu to do this later